## Observations and Insights 

In [58]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
from functools import reduce
from scipy.stats import linregress
import scipy.stats as st
import numpy as np

# Study data files
#athlete_events_path = "data_set/athlete_events.csv"
olympic_medals_path = "data_set/olympic_medals.csv"
Countrys_GDP_path = "data_set/Country_wise_GDP_from_1994_to_2017.csv"

# Read the olympics data and the study results
medals_Year = []
#athlete_events = pd.read_csv(athlete_events_path)
olympic_medals = pd.read_csv(olympic_medals_path)
Countrys_GDP = pd.read_csv(Countrys_GDP_path)
medals_Year = olympic_medals['slug_game'].str[-4:]
olympic_medals['medals_Year'] = medals_Year.astype('int64')
# Concatinate athletes name in olympic medals file to come up with full name that can be used to merge with athlete_events file 

olympic_medals

,Unnamed: 0,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_name,athlete_surname,country_name,country_code,country_tri_letter_code,medals_Year
0,0,Skeleton,pyeongchang-2018,Women,Women,GOLD,Athlete,NaN,Lizzy,YARNOLD,Great Britain,GB,GBR,2018
1,1,Skeleton,pyeongchang-2018,Women,Women,SILVER,Athlete,NaN,Jacqueline,LOELLING,Germany,DE,GER,2018
2,2,Skeleton,pyeongchang-2018,Women,Women,BRONZE,Athlete,NaN,Laura,DEAS,Great Britain,GB,GBR,2018
3,3,Skeleton,pyeongchang-2018,Men,Men,GOLD,Athlete,NaN,Sungbin,YUN,Republic of Korea,KR,KOR,2018
4,4,Skeleton,pyeongchang-2018,Men,Men,SILVER,Athlete,NaN,Nikita,TREGUBOV,Olympic Athletes from Russia,NaN,OAR,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20137,20137,Weightlifting,athens-1896,heavyweight - one hand lift men,Men,SILVER,Athlete,NaN,Viggo,JENSEN,Denmark,DK,DEN,1896
20138,20138,Weightlifting,athens-1896,heavyweight - one hand lift men,Men,BRONZE,Athlete,NaN,Alexandros,Nikolopoulos,Greece,GR,GRE,1896
20139,20139,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,GOLD,Athlete,NaN,Viggo,JENSEN,Denmark,DK,DEN,1896
20140,20140,Weightlifting,athens-1896,heavyweight - two hand lift men,Men,SILVER,Athlete,NaN,Launceston,ELLIOT,Great Britain,GB,GBR,1896


In [52]:
diff_cols = Countrys_GDP.columns.difference(olympic_medals.columns)

In [53]:
diff_cols

Index(['Country', 'GDP (in USD)', 'GDP Real (in USD)', 'GDP change (%)',
       'GDP per capita (in USD)', 'Pop. change (%)', 'Population', 'Year'],
      dtype='object')

In [56]:
#Filter out the columns that are different. You could pass in the df2[diff_cols] directly into the merge as well.
Diff_GDP_columns = Countrys_GDP[diff_cols]

#Merge the DataFrames
df_merged = pd.merge(olympic_medals, Diff_GDP_columns, right_on=['Country','Year'],left_on=['country_name','medals_Year'], how='inner')

In [57]:
df_merged

,Unnamed: 0,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_name,athlete_surname,...,country_tri_letter_code,medals_Year,Country,GDP (in USD),GDP Real (in USD),GDP change (%),GDP per capita (in USD),Pop. change (%),Population,Year
0,331,Cycling BMX,rio-2016,Individual women,Women,GOLD,Athlete,NaN,Mariana,PAJON,...,COL,2016,Colombia,282856115393,366906206489,1.96,7616,1.38,48175052,2016
1,336,Cycling BMX,rio-2016,Individual men,Men,BRONZE,Athlete,NaN,Carlos Alberto,RAMIREZ YEPES,...,COL,2016,Colombia,282856115393,366906206489,1.96,7616,1.38,48175052,2016
2,556,Athletics,rio-2016,triple jump women,Women,GOLD,Athlete,NaN,Caterine,IBARGUEN,...,COL,2016,Colombia,282856115393,366906206489,1.96,7616,1.38,48175052,2016
3,658,Weightlifting,rio-2016,62kg featherweight men,Men,GOLD,Athlete,NaN,Oscar,FIGUEROA,...,COL,2016,Colombia,282856115393,366906206489,1.96,7616,1.38,48175052,2016
4,675,Weightlifting,rio-2016,69kg men,Men,BRONZE,Athlete,NaN,Luis Javier,MOSQUERA LOZANO,...,COL,2016,Colombia,282856115393,366906206489,1.96,7616,1.38,48175052,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4868,7851,Speed skating,lillehammer-1994,5000m men,Men,BRONZE,Athlete,NaN,Rintje,RITSMA,...,NED,1994,Netherlands,374291430318,576265203487,2.96,37495,0.66,15369120,1994
4869,7872,Short Track Speed Skating,lillehammer-1994,5000m relay men,Men,BRONZE,GameTeam,Australia team,NaN,NaN,...,AUS,1994,Australia,322201314424,663034939582,3.98,37252,1.11,17798529,1994
4870,7934,Cross Country Skiing,lillehammer-1994,Combined 10km 15km pursuit men,Men,SILVER,Athlete,NaN,Vladimir,SMIRNOV,...,KAZ,1994,Kazakhstan,21250839258,64407663403,-12.60,4020,-0.98,16022222,1994
4871,7942,Cross Country Skiing,lillehammer-1994,50km men,Men,GOLD,Athlete,NaN,Vladimir,SMIRNOV,...,KAZ,1994,Kazakhstan,21250839258,64407663403,-12.60,4020,-0.98,16022222,1994


In [59]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4873 entries, 0 to 4872
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               4873 non-null   int64  
 1   discipline_title         4873 non-null   object 
 2   slug_game                4873 non-null   object 
 3   event_title              4873 non-null   object 
 4   event_gender             4873 non-null   object 
 5   medal_type               4873 non-null   object 
 6   participant_type         4873 non-null   object 
 7   participant_title        1541 non-null   object 
 8   athlete_name             4138 non-null   object 
 9   athlete_surname          4138 non-null   object 
 10  country_name             4873 non-null   object 
 11  country_code             4855 non-null   object 
 12  country_tri_letter_code  4873 non-null   object 
 13  medals_Year              4873 non-null   int64  
 14  Country                 

In [60]:
df_merged.describe()

,Unnamed: 0,medals_Year,GDP (in USD),GDP Real (in USD),GDP change (%),GDP per capita (in USD),Pop. change (%),Population,Year
count,4873.000000,4873.000000,4.873000e+03,4.873000e+03,4873.000000,4873.000000,4873.000000,4.873000e+03,4873.000000
mean,4203.403653,2005.831726,1.065870e+12,1.243354e+12,2.728728,31339.764621,0.541166,4.579118e+07,2005.831726
std,2220.571063,6.826462,1.314196e+12,1.432649e+12,2.943346,21673.973849,0.818036,7.639258e+07,6.826462
min,331.000000,1994.000000,2.195836e+08,2.916737e+08,-22.930000,197.000000,-2.600000,9.627300e+04,1994.000000
25%,2256.000000,2000.000000,1.260080e+11,1.424573e+11,1.170000,8727.000000,0.030000,9.458075e+06,2000.000000
50%,4233.000000,2006.000000,4.419961e+11,5.733236e+11,2.620000,36655.000000,0.470000,2.886073e+07,2006.000000
75%,6174.000000,2012.000000,1.635015e+12,2.060210e+12,4.000000,44999.000000,1.060000,6.150893e+07,2012.000000
max,7968.000000,2016.000000,6.203213e+12,6.052672e+12,26.850000,90864.000000,9.610000,1.324517e+09,2016.000000


In [90]:
nuu_df = df_merged[df_merged['athlete_name'].isnull()]
nuu_df_1 = df_merged[df_merged['participant_title'].isnull()]
nuu_df_3 = df_merged[df_merged['participant_title'].notnull()]
nuu_df_4 = df_merged[df_merged['country_tri_letter_code'].isnull()]

In [92]:
nuu_df_4['country_tri_letter_code'].unique()

array([], dtype=object)

In [89]:
df_merged

,Unnamed: 0,discipline_title,slug_game,event_title,event_gender,medal_type,participant_type,participant_title,athlete_name,athlete_surname,...,country_tri_letter_code,medals_Year,Country,GDP (in USD),GDP Real (in USD),GDP change (%),GDP per capita (in USD),Pop. change (%),Population,Year
0,331,Cycling BMX,rio-2016,Individual women,Women,GOLD,Athlete,NaN,Mariana,PAJON,...,COL,2016,Colombia,282856115393,366906206489,1.96,7616,1.38,48175052,2016
1,336,Cycling BMX,rio-2016,Individual men,Men,BRONZE,Athlete,NaN,Carlos Alberto,RAMIREZ YEPES,...,COL,2016,Colombia,282856115393,366906206489,1.96,7616,1.38,48175052,2016
2,556,Athletics,rio-2016,triple jump women,Women,GOLD,Athlete,NaN,Caterine,IBARGUEN,...,COL,2016,Colombia,282856115393,366906206489,1.96,7616,1.38,48175052,2016
3,658,Weightlifting,rio-2016,62kg featherweight men,Men,GOLD,Athlete,NaN,Oscar,FIGUEROA,...,COL,2016,Colombia,282856115393,366906206489,1.96,7616,1.38,48175052,2016
4,675,Weightlifting,rio-2016,69kg men,Men,BRONZE,Athlete,NaN,Luis Javier,MOSQUERA LOZANO,...,COL,2016,Colombia,282856115393,366906206489,1.96,7616,1.38,48175052,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4868,7851,Speed skating,lillehammer-1994,5000m men,Men,BRONZE,Athlete,NaN,Rintje,RITSMA,...,NED,1994,Netherlands,374291430318,576265203487,2.96,37495,0.66,15369120,1994
4869,7872,Short Track Speed Skating,lillehammer-1994,5000m relay men,Men,BRONZE,GameTeam,Australia team,NaN,NaN,...,AUS,1994,Australia,322201314424,663034939582,3.98,37252,1.11,17798529,1994
4870,7934,Cross Country Skiing,lillehammer-1994,Combined 10km 15km pursuit men,Men,SILVER,Athlete,NaN,Vladimir,SMIRNOV,...,KAZ,1994,Kazakhstan,21250839258,64407663403,-12.60,4020,-0.98,16022222,1994
4871,7942,Cross Country Skiing,lillehammer-1994,50km men,Men,GOLD,Athlete,NaN,Vladimir,SMIRNOV,...,KAZ,1994,Kazakhstan,21250839258,64407663403,-12.60,4020,-0.98,16022222,1994
